In [1]:
from transformers import BartTokenizer, BartForConditionalGeneration
import pandas as pd
from datasets import Dataset
from tqdm import tqdm
import sacrebleu


model_name = "facebook/bart-large"
tokenizer = BartTokenizer.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name)


c:\Users\gheto\miniconda3\envs\aromanianLLM\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\gheto\miniconda3\envs\aromanianLLM\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [2]:

# Load your translations.csv
df = pd.read_csv("../dataset/nllb_corpus_train.csv")
eval_df = pd.read_csv("../dataset/nllb_corpus_test.csv")
# Rename columns if needed
df.columns = ['ro', 'rup']
# Convert to Hugging Face Dataset format
dataset = Dataset.from_pandas(df)
eval_dataset = Dataset.from_pandas(eval_df)


In [3]:
def preprocess_function(examples, max_length=128):
    inputs = examples['ro']
    targets = examples['rup']
    model_inputs = tokenizer(inputs, max_length=max_length, truncation=True, padding="max_length")
    
    # Tokenize the target (Aromanian) with the target tokenizer
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_length, truncation=True, padding="max_length")

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Apply the preprocessing to the entire dataset
tokenized_dataset = dataset.map(preprocess_function, batched=True)
tokenized_eval_dataset = eval_dataset.map(preprocess_function, batched=True)


Map:   0%|          | 0/27033 [00:00<?, ? examples/s]c:\Users\gheto\miniconda3\envs\aromanianLLM\Lib\site-packages\transformers\tokenization_utils_base.py:4126: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Map: 100%|██████████| 3004/3004 [00:00<00:00, 8516.78 examples/s]


In [4]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",  
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    save_steps=1000,
    save_total_limit=2,
    learning_rate=5e-5,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,  
    eval_dataset=tokenized_eval_dataset,  
)

trainer.train()


c:\Users\gheto\miniconda3\envs\aromanianLLM\Lib\site-packages\transformers\training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
  0%|          | 0/10140 [00:00<?, ?it/s]c:\Users\gheto\miniconda3\envs\aromanianLLM\Lib\site-packages\transformers\models\bart\modeling_bart.py:496: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
  5%|▍         | 500/10140 [02:34<48:50,  3.29it/s]

{'loss': 1.1083, 'grad_norm': 1.3328168392181396, 'learning_rate': 4.7534516765285995e-05, 'epoch': 0.15}


 10%|▉         | 1000/10140 [05:03<45:34,  3.34it/s]Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}


{'loss': 0.1957, 'grad_norm': 0.7041230797767639, 'learning_rate': 4.5069033530571994e-05, 'epoch': 0.3}


 15%|█▍        | 1500/10140 [07:38<43:13,  3.33it/s]  

...

 99%|█████████▊| 10000/10140 [52:34<00:43,  3.23it/s]Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}


{'loss': 0.1106, 'grad_norm': 0.8834843635559082, 'learning_rate': 6.903353057199211e-07, 'epoch': 2.96}


100%|██████████| 10140/10140 [53:25<00:00,  3.86it/s]Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}
                                                     
100%|██████████| 10140/10140 [53:59<00:00,  3.13it/s]

{'eval_loss': 0.13884691894054413, 'eval_runtime': 28.8257, 'eval_samples_per_second': 104.213, 'eval_steps_per_second': 13.044, 'epoch': 3.0}
{'train_runtime': 3239.2661, 'train_samples_per_second': 25.036, 'train_steps_per_second': 3.13, 'train_loss': 0.18929129023053473, 'epoch': 3.0}


TrainOutput(global_step=10140, training_loss=0.18929129023053473, metrics={'train_runtime': 3239.2661, 'train_samples_per_second': 25.036, 'train_steps_per_second': 3.13, 'total_flos': 2.1968752015835136e+16, 'train_loss': 0.18929129023053473, 'epoch': 3.0})

In [7]:
def translate(sentence):
    inputs = tokenizer([sentence], return_tensors="pt", max_length=128, truncation=True)
    inputs = inputs.to(model.device)
    output = model.generate(**inputs)
    return tokenizer.decode(output[0], skip_special_tokens=True)

ro_sentence = "Mahnit adanc, porni spre casa, rusinat de neizbanda lui, de parca se ducea la taierea capului."
print(translate(ro_sentence))  


Mahnit adancu, cari s-dutea,


In [ ]:
df_ro_rup_test = pd.read_csv("../dataset/nllb_corpus_test.csv")
df_ro_rup_test['ro_pred'] = ''
df_ro_rup_test['rup_pred'] = ''

test_len = len(df_ro_rup_test)
test_len = 200
for i in tqdm(range(0, test_len)):
    ro_texts = df_ro_rup_test.loc[i, 'ro']
    
    if ro_texts:
        df_ro_rup_test.loc[i, 'rup_pred'] = translate(ro_texts)

In [15]:
bleu_calc = sacrebleu.BLEU()
chrf_calc = sacrebleu.CHRF()
print(bleu_calc.corpus_score(df_ro_rup_test['rup'][:200].to_list(), [df_ro_rup_test['rup_pred'][:200].to_list()]))
print(chrf_calc.corpus_score(df_ro_rup_test['rup'][:200].to_list(), [df_ro_rup_test['rup_pred'][:200].to_list()]))

BLEU = 1.28 9.5/4.7/0.3/0.2 (BP = 1.000 ratio = 1.381 hyp_len = 370 ref_len = 268)
chrF2 = 19.90
